<a href="https://colab.research.google.com/github/salmakhale/DL-computer-vision/blob/main/cnn_ecg_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
shayanfazeli_heartbeat_path = kagglehub.dataset_download('shayanfazeli/heartbeat')

print('Data source import complete.')


## ECG Heartbeat Classification Datset
This notebook involves the making of machine learning models to classify the given data of obtained as an heartbeat ECG into differen classes. We'll undergo machine learning processes to classify them. AS given in the dataset,  we are given 5 different classes of heartbeat as [N:0, S:1, V:2 , F:3, Q:4]
* N: Non-Ectopic Beats
* S: Superventrical Ectopic Beats
* V: Ventricular Ectopic Beats
* F: Fusion Beats
* Q: Unknown Beats

The **CNN Algotithm** that we'll implement will classigy the given heartbeat into one of these classes

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import os, tqdm, re, time, itertools, sys
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Loading the Data
The first step is to load the data in our memory. We'll load all data provided to us in our notebook, and then start the machine learning process

In [ ]:
# loading all data files into memory
start = time.time()

data_train = pd.read_csv('../input/heartbeat/mitbih_train.csv', header=None)
data_test = pd.read_csv('../input/heartbeat/mitbih_test.csv', header=None)
abnormal = pd.read_csv('../input/heartbeat/ptbdb_abnormal.csv', header=None)
normal = pd.read_csv('../input/heartbeat/ptbdb_normal.csv', header=None)

end = time.time()
print('Time taken: %.3f seconds' % (end-start))

print('Data loaded........')

In [ ]:
normal = normal.drop([187], axis=1)
abnormal = abnormal.drop([187], axis=1)

## EDA (Exploratory Data Analysis)
In this step, we will undergo an EDA (Exploratory Data Analysis) to get brief understanding of our data. We are given a data concerned with the ECG of a patient, classified into normal and abnormal classes. We'll make some plots to see the variations in the heart rate of a patient with normal and abnormal ECG.  

In [ ]:
data_train.isnull().sum().to_numpy()

In [ ]:
# checking the dataset shape
abnormal.shape, normal.shape

We have a total of **10506** rows and **188** columns for abormal & **4045** rows and **188** columns of a normal ECG in our data

In [ ]:
# view first 4 rows of data
data_train.head(4)

As it can be seen, the data is composed of columns (features) that contain the floating point numbers that represent the heart rate.

In [ ]:
# checking the columns
data_train.columns

In [ ]:
abnormal.shape, normal.shape

In [ ]:
# view first 2 rows of abnormal ECG data
abnormal.head(2)

In [ ]:
# view first 2 rows of normal data|
normal.head(2)

In [ ]:
flatten_y = abnormal.values
flatten_y = flatten_y[:, 5:70].flatten()
flatten_y

### Data Visualization
For better comprehension, we'll plot the data of normal and abnormal ECG rate to see how the curves look like. Given below are some plots of normal and abnormal ECG rate.

### Abormal ECG Visualization
Below are some plots showing the ECG Curve of those persons who have an abnormal ECG rate

In [ ]:
plt.figure(figsize=(15, 3))
plt.title('ECG Visualization of Abormal Persons')
plt.subplot(1, 5, 1)
plt.plot(abnormal.values[0][5:50])
plt.subplot(1, 5, 2)
plt.plot(abnormal.values[10][5:50])
plt.subplot(1, 5, 3)
plt.plot(abnormal.values[20][5:50])
plt.subplot(1, 5, 4)
plt.plot(abnormal.values[40][5:50])
plt.subplot(1, 5, 5)
plt.plot(abnormal.values[44][5:50])

## Normal ECG Visualization
Below are the graphs showing the ECG rate of normal persons

In [ ]:
plt.figure(figsize=(15, 3))
plt.title('ECG Visualization of Normal Persons')
plt.subplot(1, 5, 1)
plt.plot(normal.values[0][5:50])
plt.subplot(1, 5, 2)
plt.plot(normal.values[10][5:50])
plt.subplot(1, 5, 3)
plt.plot(normal.values[20][5:50])
plt.subplot(1, 5, 4)
plt.plot(normal.values[40][5:50])
plt.subplot(1, 5, 5)
plt.plot(normal.values[77][5:50])

In [ ]:
fig, axs = plt.subplots(4)
fig.suptitle('Vertically stacked ECG plots of Normal People')
axs[0].plot(normal.values[10][1:70])
axs[1].plot(normal.values[55][1:70])
axs[2].plot(normal.values[87][1:70])
axs[3].plot(normal.values[98][1:70])

In [ ]:
# viewing the distribution of beats in our dataset
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(data_train[187].value_counts(), labels=['Non-Ectopic Beats', 'Superventrical Ectopic', 'Ventricular Beats',
                                                'Unknown', 'Fusion Beats'], colors=['blue', 'magenta', 'cyan',
                                                                                   'red', 'grey'])
p = plt.gcf()
p.gca().add_artist(circle)

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(10, 5))
plt.plot(data_train.iloc[0, 0:187])

## Conclusion:
We can conclude from above figures that the persons having normal ECG rate, the figures are following a **bell-curve** pattern. The ECG of abnormal persons show othe types of curves. We'll use this information to make our machine learning model for classification.

## Data Preprocessing
At this stage, we'll undergo some data preprocessing process to see if the data needs to be cleaned. Cleaned data is requried for model fitting in next phases.

In [ ]:
# making the class labels for our dataset
data_1 = data_train[data_train[187] == 1]
data_2 = data_train[data_train[187] == 2]
data_3 = data_train[data_train[187] == 3]
data_4 = data_train[data_train[187] == 4]

sns.set_style('darkgrid')
plt.figure(figsize=(10, 5))
plt.plot(data_train.iloc[0, 0:187], color='green', label='Normal Heartbeats')
plt.plot(data_1.iloc[0, 0:187], color='blue', label='Supraventricular Heartbeats')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(10, 5))
plt.plot(data_train.iloc[0, 0:187], color='red', label='Normal Heartbeats')
plt.plot(data_4.iloc[0, 0:187], color='grey', label='Unknown Heartbeats')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()

In [ ]:
y_abnormal = np.ones(abnormal.shape[0])
y_abnormal = pd.DataFrame(y_abnormal)

y_normal = np.zeros(normal.shape[0])
y_normal = pd.DataFrame(y_normal)

# merging the original dataframe
X = pd.concat([abnormal, normal], sort=True)
y = pd.concat([y_abnormal, y_normal], sort=True)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# checking if there are some null values in data
normal.isnull().sum().to_numpy()

In [ ]:
# checking if there are some null values in abnormal patient data
abnormal.isnull().sum().to_numpy()

The output of above cell shows that there are no null values in our data, and the data can now be used for model fitting. We have two types of datasets, normal and abnormal, and they'll be used for model fitting.  

## Data Argumentation
Since our data in biased, we need to use data argumentation on it so that we can remove bias from data and make equal distributions.

In [ ]:
from sklearn.utils import resample
data_1_resample = resample(data_1, n_samples=20000,
                           random_state=123, replace=True)
data_2_resample = resample(data_2, n_samples=20000,
                           random_state=123, replace=True)
data_3_resample = resample(data_3, n_samples=20000,
                           random_state=123, replace=True)
data_4_resample = resample(data_4, n_samples=20000,
                           random_state=123, replace=True)
data_0 = data_train[data_train[187] == 0].sample(n=20000, random_state=123)

In [ ]:
train_dataset = pd.concat([data_0, data_1_resample, data_2_resample, data_3_resample,
                          data_4_resample])

In [ ]:
# viewing the distribution of beats in our dataset
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(train_dataset[187].value_counts(), labels=['Non-Ectopic Beats', 'Superventrical Ectopic', 'Ventricular Beats',
                                                'Unknown', 'Fusion Beats'], colors=['blue', 'magenta', 'cyan',
                                                                                   'red', 'grey'])
p = plt.gcf()
p.gca().add_artist(circle)

## Making X & Y Variables

In [ ]:
target_train = train_dataset[187]
target_test = data_test[187]
target_train.unique()

In [ ]:
y_train = to_categorical(target_train)
y_test = to_categorical(target_test)
y_train[:4]

## Data Splicing
This stage involves the data split into train & test sets. The training data will be used for training our model, and the testing data will be used to check the performance of model on unseen dataset. We're using a split of **80-20**, i.e., **80%** data to be used for training & **20%** to be used for testing purpose.  

In [ ]:
# making train & test splits
X_train = train_dataset.iloc[:, :-1].values
X_test = data_test.iloc[:, :-1].values

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Applying the Model
We are making use of following models to make our classification:
* Random Forest Classification
* Support Vector Machines (SVM)
* Convolutional Neural Network (CNN)

Steps:
* We will instantiate the model
* After intantiation, the model will be fit to training data
* After then, the model will be tested on useen data to make predictions

## Convolutional Neural Network (CNN)
We will apply the CNN algorithm to our data to generate prediction results.
First, we need to reshape our data for CNN. We will use 1-dimensional CNN for our model, reshaping our data as per the dimensins of our CNN>  

In [ ]:
X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
X_test = X_test.reshape(len(X_test), X_test.shape[1], 1)
X_train.shape, X_test.shape

In [ ]:
# making the deep learning function
def model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu',
                    padding='same', input_shape=(187, 1)))
    model.add(BatchNormalization())

    # adding a pooling layer
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))

    model.add(Conv1D(filters=64, kernel_size=6, activation='relu',
                    padding='same', input_shape=(187, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))

    model.add(Conv1D(filters=64, kernel_size=6, activation='relu',
                    padding='same', input_shape=(187, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides=2, padding='same'))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = model()
model.summary()

In [ ]:
logger = CSVLogger('logs.csv', append=True)
his = model.fit(X_train, y_train, epochs=50, batch_size=32,
          validation_data=(X_test, y_test), callbacks=[logger])

In [ ]:
model.evaluate(X_test, y_test)

## Graphical Visualization of Predictions

In [ ]:
history = his.history
history.keys()

In [ ]:
epochs = range(1, len(history['loss']) + 1)
acc = history['accuracy']
loss = history['loss']
val_acc = history['val_accuracy']
val_loss = history['val_loss']

plt.figure(figsize=(10, 5))
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(epochs, acc, label='accuracy')
plt.plot(epochs, val_acc, label='val_acc')
plt.legend()

plt.figure(figsize=(10, 5))
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(epochs, loss, label='loss', color='g')
plt.plot(epochs, val_loss, label='val_loss', color='r')
plt.legend()

In [ ]:
y_pred = model.predict(X_test)
y_hat = np.argmax(y_pred, axis = 1)
confusion_matrix(np.argmax(y_test, axis = 1), y_hat)

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(confusion_matrix(np.argmax(y_test, axis = 1), y_hat), annot=True, fmt='0.0f', cmap='RdPu')

### 1. Random Forest Classifier
In this section, we use Random Forest Classifier to fit to our training dataset & predict results.

In [ ]:
# instantiate the classifier and fit to training data
start = time.time()

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

end = time.time()
print('Time Taken: %.3f seconds' % (end-start))

The model has been successfully fit to our training data. Let's check its performance on test set.

In [ ]:
# viewing params of random forest
rf.get_params()

In [ ]:
# making predictions on test set
start = time.time()

y_pred = rf.predict(X_test)

end = time.time()
print('Time Taken: %.3f seconds' % (end-start))

y_pred[:10]

In [ ]:
# check accuracy
print('Accuracy on train data: %.4f' % rf.score(X_train, y_train))
print('Accuracy on test data %.4f' % rf.score(X_test, y_test))

### 2. Support Vector Machine (SVM)
In this section, we'll fit SVM on our training dataset and check its performance on test data.

In [ ]:
# instantiate ^ fit SVM to train data
svc = SVC()
svc.fit(X_train, y_train)

Model has been successfully fit to train data

In [ ]:
# check parameters of SVM
svc.get_params()

In [ ]:
# predict on test set
y_pred_svc = svc.predict(X_test)
y_pred_svc[:5]

In [ ]:
# check accuracy of SVM
print('Acc train %.4f' % svc.score(X_train, y_train))
print('Acc test %.4f' % svc.score(X_test, y_test))

As given, we are getting **92%** accuracy on training & **90%** accuracy on test dataset using **Support Vector Machines (SVM)**.  

## Prediction Results
In this section, we'll visualize the results obtained by **SVM** & **Random Forest**. The results are:
* Random Forest got an accuracy of **99%** on training data and **97%** on test data
* SVM got an accuracy of **92%** on train data and **90%** on test data
* We'll also plot confusion matrices of respective classifiers

In [ ]:
acc_rf = rf.score(X_test, y_test)
acc_svm = rf.score(X_test, y_test)
classifiers = ['Random Forest', 'SVM']
plt.title('Accuracy of SVM & Random Forest on Test Data')
plt.ylabel('Accuracy')
plt.bar(classifiers, [acc_rf, acc_svm])

In [ ]:
# print classification report of SVM anf RF
print('Classification Report of Random Forest:')
print('')
print(classification_report(y_test, y_pred))

In [ ]:
# print classification report of SVM anf RF
print('Classification Report of SVM:')
print('')
print(classification_report(y_test, y_pred_svc))

## Confusion Matrices
Below are the confusion matrices obtained as a result of classification from Random Forest & SVM:

### Confuion Matrix for Random Forest

In [ ]:
# confusion matrix for Random Forest
plot_confusion_matrix(rf, X_test, y_test)

### Confuion Matrix for SVM

In [ ]:
# confusion matrix for Random Forest
plot_confusion_matrix(svc, X_test, y_test)